In [1]:
import numpy as np
import scipy.sparse as sparse
import random
import hashlib

# Load in Data

In [2]:
data = np.load('Data/user_movie_rating.npy')[:,0:2]

In [30]:
col = data[:, 0]
row = data[:, 1]

n_col = col.max() + 1
n_row = row.max() + 1 

M = sparse.coo_matrix((np.ones(len(data)), (row, col)), shape=(n_row, n_col)).tocsr()
small_M  = M[:,0:1000]

In [31]:
perm = 100 
perm_seeds = []
row_len,_ = small_M.get_shape()
seed = 100000
for i in range(perm):
    perm_seeds.append(int(hashlib.sha256(str(seed+i).encode()).hexdigest(),16)%4294967295)


In [37]:
from tqdm import tqdm

matrix = small_M
##TODO: Implement multithreading so we can calculate the seeds in parallel


signature = []
for seed in tqdm(perm_seeds):
    row_signature = []
    #Shuffle indexes to get random permutation
    random.seed(seed)
    index_perm = np.arange(matrix.shape[0])
    np.random.shuffle(index_perm)
    #Initialize permutated matrix (maybe not nessesary just acces element by element in the perm index)
    perm_M = matrix[index_perm,1:] #0 index is empty
    row_signature = np.full(perm_M.shape[1], np.inf, dtype=float)#initialize as infinite
    user_ids = np.arange(start=1, stop=matrix.shape[1])
    # Run varaible to get the right index
    row_idx = 0
    while perm_M.shape[1] != 0:
        matches = perm_M[row_idx,:].nonzero()[1]
        non_matches = np.setdiff1d(np.arange(perm_M.shape[1]), matches)
        row_signature[user_ids[matches]-1] = row_idx
        perm_M = perm_M[:,non_matches]
        user_ids = user_ids[non_matches]
        row_idx +=1
    signature.append(row_signature)
        

        


100%|██████████| 100/100 [00:29<00:00,  3.34it/s]


17771

In [62]:
M.shape

(17771, 103704)

103704

In [33]:
np.array(signature)

array([[ 3., 49.,  1., ..., 41.,  3.,  3.],
       [10., 12.,  4., ..., 20., 91., 24.],
       [40., 23., 44., ..., 44., 91., 23.],
       ...,
       [36.,  6., 83., ...,  1., 16., 75.],
       [23., 11., 16., ..., 36., 53., 53.],
       [70., 70., 61., ..., 37., 26., 23.]], shape=(100, 999))

In [7]:
small_M[:,1]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 626 stored elements and shape (17771, 1)>

In [45]:
6*100/60

10.0

In [ ]:
def LSH():
    pass
# Devide Matrix into b bands and r rows
# implement hash table to check for identity in linear time

        


_IncompleteInputError: incomplete input (2385955972.py, line 3)

In [13]:
index = np.arange(np.shape(small_M)[0])
print(index)
np.random.shuffle(index)
print(small_M[index, :])

[    0     1     2 ... 17768 17769 17770]
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 628113 stored elements and shape (17771, 1000)>
  Coords	Values
  (0, 550)	1.0
  (0, 781)	1.0
  (2, 95)	1.0
  (2, 173)	1.0
  (2, 207)	1.0
  (2, 469)	1.0
  (2, 795)	1.0
  (2, 895)	1.0
  (3, 103)	1.0
  (3, 352)	1.0
  (3, 628)	1.0
  (4, 299)	1.0
  (4, 673)	1.0
  (4, 755)	1.0
  (4, 836)	1.0
  (5, 110)	1.0
  (5, 166)	1.0
  (5, 207)	1.0
  (5, 526)	1.0
  (5, 779)	1.0
  (6, 204)	1.0
  (6, 327)	1.0
  (6, 556)	1.0
  (6, 783)	1.0
  (6, 996)	1.0
  :	:
  (17764, 985)	1.0
  (17764, 986)	1.0
  (17764, 991)	1.0
  (17764, 998)	1.0
  (17767, 81)	1.0
  (17767, 118)	1.0
  (17767, 130)	1.0
  (17767, 148)	1.0
  (17767, 180)	1.0
  (17767, 244)	1.0
  (17767, 269)	1.0
  (17767, 385)	1.0
  (17767, 500)	1.0
  (17767, 597)	1.0
  (17767, 628)	1.0
  (17767, 855)	1.0
  (17767, 867)	1.0
  (17767, 912)	1.0
  (17767, 960)	1.0
  (17769, 106)	1.0
  (17769, 200)	1.0
  (17769, 301)	1.0
  (17769, 693)	1.0
  (17769, 793)	1

In [14]:
index

array([15050,  6862,  7205, ...,  4307,  3482,  6790], shape=(17771,))

array([    0,     1,     2, ..., 17768, 17769, 17770], shape=(17771,))